<a href="https://colab.research.google.com/github/ravi-kr/Universe/blob/main/algorithmic_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Building an equal weight NIFTY 50 Index Fund

In [ ]:
import pandas as pd
import math

In [ ]:
#https://www.nseindia.com/market-data/live-equity-market
stock_df = pd.read_csv('Nifty-50-stock.csv', usecols = [0,5], skiprows = 2, names = ['Ticker', 'Stock Price (Rs)'])
stock_df.head()

In [ ]:
#https://www1.nseindia.com/products/content/equities/indices/homepage_indices_monthly.htm
market_cap_df = pd.read_csv('Nifty-50-market-cap.csv', usecols = [1, 5], skiprows = 3, names = ['Ticker', 'Market Capitalization (Cr)'], nrows = 50)
market_cap_df.head()

In [ ]:
final_dataframe = pd.merge(stock_df,market_cap_df,on='Ticker')

In [ ]:
final_dataframe.head()

In [ ]:
portfolio_size = input('Enter the value of your portfolio: ')

In [ ]:
val = float(portfolio_size)
position_size = val/len(final_dataframe.index)
print(position_size)

In [ ]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(final_dataframe.loc[i, 'Stock Price (Rs)'].replace(',', '')))
    
final_dataframe

In [ ]:
final_dataframe.to_csv('recommended trades new.csv')

In [ ]:
# Building an equal weight NIFTY Index Fund with Quantitative Momentum Strategy

In [ ]:
# https://investyadnya.in/

In [ ]:
from nsepy import get_history
from datetime import date
data = get_history(symbol='Reliance',
                    start=date(2021,4,1), 
                    end=date(2022,4,1))
data

In [ ]:
return_data = pd.DataFrame()
for Ticker in final_dataframe['Ticker']:
    data = get_history(symbol=Ticker,
                    start=date(2021,4,1), 
                    end=date(2022,4,1))
    print('Ticker')
    return_data = return_data.append(pd.DataFrame([[data.loc[date(2021,4,1), 'Symbol'], data.loc[date(2021,4,1), 'Close'], data.loc[date(2022,4,1), 'Close']]], columns = ['Ticker', '2021-04-01', '2022-04-01']),ignore_index=True)
return_data

In [ ]:
return_data

In [ ]:
return_data['one year return'] = ((return_data['2022-04-01'] - return_data['2021-04-01']) / return_data['2021-04-01'])
return_data['one year return'] = return_data['one year return'].round(decimals = 4)
#return_data['one year return'] = return_data['one year return'].astype(str) + '%'
return_data

In [ ]:
final_return_data = return_data[['Ticker', '2022-04-01', 'one year return']]
final_return_data.columns = ['Ticker', 'Stock Price (Rs)', 'One Year Return']
final_return_data = final_return_data.sort_values('One Year Return', ascending = False)

In [ ]:
final_return_data = final_return_data[:15]

In [ ]:
final_return_data = final_return_data.reset_index(drop=True)

In [ ]:
portfolio_size = input('Enter the value of your portfolio: ')

In [ ]:
val = float(portfolio_size)
position_size = val/len(final_return_data.index)
print(position_size)

In [ ]:
for i in range(len(final_return_data.index)):
    final_return_data.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(final_return_data.iloc[i, 1]))
    
final_return_data

In [ ]:
#Building a better (and more realistic) momentum strategy

In [ ]:
return_data_hqm = pd.DataFrame()
for Ticker in final_dataframe['Ticker']:
    data = get_history(symbol=Ticker,
                    start=date(2021,4,1), 
                    end=date(2022,4,1))
    print(Ticker)
    return_data_hqm = return_data_hqm.append(pd.DataFrame([[data.loc[date(2021,4,1), 'Symbol'], data.loc[date(2021,4,1), 'Close'],data.loc[date(2021,10,1), 'Close'],data.loc[date(2022,1,3), 'Close'],data.loc[date(2022,3,2), 'Close'], data.loc[date(2022,4,1), 'Close']]], columns = ['Ticker', '2021-04-01', '2021-10-01', '2022-01-01', '2022-03-01', '2022-04-01']),ignore_index=True)
return_data_hqm

In [ ]:
return_data_hqm['one year return'] = ((return_data_hqm['2022-04-01'] - return_data_hqm['2021-04-01']) / return_data_hqm['2021-04-01']).round(decimals = 4)
return_data_hqm['six month return'] = ((return_data_hqm['2022-04-01'] - return_data_hqm['2021-10-01']) / return_data_hqm['2021-04-01']).round(decimals = 4)
return_data_hqm['three month return'] = ((return_data_hqm['2022-04-01'] - return_data_hqm['2022-01-01']) / return_data_hqm['2021-04-01']).round(decimals = 4)
return_data_hqm['one month return'] = ((return_data_hqm['2022-04-01'] - return_data_hqm['2022-03-01']) / return_data_hqm['2021-04-01']).round(decimals = 4)

return_data_hqm

In [ ]:
from scipy.stats import percentileofscore as score
time_periods = [
                'One Year',
                'Six Month',
                'Three Month',
                'One Month'
                ]

for row in final_return_data_hqm.index:
    for time_period in time_periods:
        final_return_data_hqm.loc[row, f'{time_period} Return Percentile'] = score(final_return_data_hqm[f'{time_period} Return'], final_return_data_hqm.loc[row, f'{time_period} Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(final_return_data_hqm[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
final_return_data_hqm

In [ ]:
from statistics import mean

for row in final_return_data_hqm.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(final_return_data_hqm.loc[row, f'{time_period} Return Percentile'])
    final_return_data_hqm.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [ ]:
final_return_data_hqm

In [ ]:
final_return_data_hqm = final_return_data_hqm.sort_values(by = 'HQM Score', ascending = False)
final_return_data_hqm = final_return_data_hqm[:15]
final_return_data_hqm = final_return_data_hqm.reset_index(drop=True)

In [ ]:
final_return_data_hqm

In [ ]:
portfolio_size = input('Enter the value of your portfolio: ')

In [ ]:
val = float(portfolio_size)
position_size = val/len(final_return_data_hqm.index)
print(position_size)

In [ ]:
for i in range(len(final_return_data_hqm.index)):
    final_return_data_hqm.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(final_return_data_hqm.iloc[i, 1]))
    
final_return_data_hqm

In [ ]:
# Quantitave Value Investing Strategy

In [ ]:
#https://www.moneycontrol.com/markets/indian-indices/

In [ ]:
Stock|LTP|PE_ratio
Adani_Ports|840.5|28.58
Apollo_Hospital|4744.45|270.27
Asian_Paints|3080.65|77.53
Axis_Bank|793.05|28.83
Bajaj_Auto|3697.3|21.86
Bajaj_Finance|7328.45|69.99
Bajaj_Finserv|16090.05|34.42
BPCL|379|5.23
Bharti_Airtel|740.65|-18.36
Britannia|3349.65|46.82
Cipla|1016.6|27.33
Coal_India|186.6|6.32
Divis_Labs|4499.25|48.47
Dr_Reddys_Labs|4299.2|38.38
Eicher_Motors|2491.45|52.82
Grasim|1765.95|22.12
HCL_Tech|1125.1|23.93
HDFC_Bank|1464.95|25.81
HDFC_Life|564.85|103.29
Hero_Motocorp|2274.4|19.95
Hindalco|546.75|20.87
HUL|2152.15|71.45
HDFC|2378.5|23.66
ICICI_Bank|762.25|21.35
IndusInd_Bank|983.4|23.84
Infosys|1748.55|29.99
ITC|269.55|20.42
JSW_Steel|766.35|14.23
Kotak_Mahindra|1779.6|34.69
Larsen|1760.8|17.2
M&M|866.1|48.7
Maruti_Suzuki|7469.5|47.21
Nestle|18382.4|0
NTPC|153.95|7.17
ONGC|174.25|7.91
Power_Grid_Corp|229.5|7.03
Reliance|2551.65|26.23
SBI_Life_Insura|1146.9|60.5
Shree_Cements|25363.55|46.51
SBI|517.7|14.51
Sun_Pharma|936.75|49.4
TCS|3661.95|36.65
TATA_Cons._Prod|817.6|68.7
Tata_Motors|431.05|-8.16
Tata_Steel|1319.5|12.73
Tech_Mahindra|1410.45|19.58
Titan_Company|2461.35|142.16
UltraTechCement|6730.05|35.58
UPL|818.65|17.63
Wipro|559.2|21.67

In [ ]:
pe_data = pd.read_csv('pe_ratio_data_nse50.txt', delimiter = '|')
pe_data

In [ ]:
pe_data.sort_values('PE_ratio', inplace = True)
pe_data = pe_data[pe_data['PE_ratio'] > 0]
pe_data = pe_data[:15]
pe_data.reset_index(inplace = True)
pe_data.drop('index', axis=1, inplace = True)

In [ ]:
pe_data

In [ ]:
portfolio_size = input('Enter the value of your portfolio: ')

In [ ]:
val = float(portfolio_size)
position_size = val/len(final_return_data_hqm.index)
print(position_size)

In [ ]:
for i in range(len(pe_data.index)):
    pe_data.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(pe_data.iloc[i, 1]))
    
pe_data

In [ ]:
'''
pe_ratio
pb_ratio
ps_ratio
ev_to_ebitda
ev_to_gross_profit
'''

In [ ]:
S_No|Name|CMP_(Rs)|price_to_earnings|price_to_sales|EV_(Cr)|ev_to_ebitda|price_to_book_value|Net_Profit_(Cr)|ROCE % 
1|Suumaya_Indust.|107.2|0.63|0.04|918.66|0.86|0.56|953.6|155.12
2|P._H._Capital|113.3|1.86|0.43|35.42|1.49|1.06|18.24|112.75
3|Bedmutha_Indus.|86.2|1.1|0.46|507.22|1.57|3.54|251.77|71.18
4|Chandra_Pr._Intl|236.55|3.43|0.16|88.02|2.49|2.92|25.5|42.74
5|INEOS_Styrolut.|1092.6|4.99|0.88|1542.18|2.67|1.84|384.88|42.45
6|Key_Corp|50.45|3.3|3.08|29.97|3.27|0.74|9.15|39.41
7|Andhra_Petrochem|118.4|4|1.06|935.28|2.55|2.29|251.53|36.78
8|Taparia_Tools|11.57|0.06|0.01|1.78|0.02|0.01|60.69|34.33
9|Prism_Finance|38.25|1.73|1.58|25.54|1.7|1.44|14.35|33.84
10|Kesar_Enterprise|103.55|1.26|0.21|128.79|0.99|0.98|82.57|32.9
11|NMDC|167.6|4.72|1.89|41001.42|2.81|1.39|10397.97|29.59
12|LKP_Finance|106.1|2.46|1.42|229.94|3.8|0.44|54.22|29.57
13|HB_Stockholdings|52.15|2.43|2.06|35.11|2.12|0.56|15.29|27.94
14|SPV_Global|12.5|0.12|0|25.21|0.35|0.02|19.61|27.85
15|Vibrant_Glo._Cap|55.55|3.16|0.49|205.48|3.94|1.28|46.01|27.07
16|Indsoya|7.87|3.1|0.12|0.24|2|0.27|0.1|26.34
17|B_&_A|284.7|3.59|0.3|122.29|2.86|0.79|24.6|23.99
18|Dhoot_Indl.Fin|92.3|2.04|2.75|130.95|3.29|0.18|28.6|23.06
19|Moongipa_Capital|23.7|3.39|2.69|7.23|2.42|1.52|2.13|22.59
20|Wallfort_Fin.|54.95|2.92|1.6|37.48|1.7|0.43|18.23|20.68
21|Rana_Sugars|36.3|3.54|0.39|707.89|3.43|1.3|141.84|20.45
22|21st_Cent._Mgmt.|30.65|1.33|0.12|19.95|0.82|0.51|24.27|20.06
23|Tata_Steel_Long|802.2|3.5|0.63|4760.33|2.77|1.39|1034.93|19.23
24|Dhunseri_Invest.|744.9|2.26|2.13|464.43|0.96|0.16|201.31|19.16
25|Dhunseri_Vent.|254.7|2.49|4.71|904.4|1.96|0.48|358.14|18.91
26|Ambalal_Sarabhai|36.3|4.02|1.46|269.35|3.34|2.54|55.23|18.24
27|Interactive_Fin|8.95|2.35|4.82|1.4|1.05|0.46|1.15|14.72
28|DCM_Nouvelle|281.75|4.18|0.62|578.83|3.05|2.06|125.98|14.48
29|Octavius_Plant.|65|4.19|1.17|20.7|4.01|1.46|4.65|14.22
30|Keynote_Finan.|109.8|4.48|2.78|67.08|3.19|0.8|17.21|13.93
31|Thakkers_Develp.|110|3.44|2.21|99|3.09|0.81|28.81|13.2
32|Saraswati_Commer|2869.95|4.97|4.14|295.52|4.26|0.79|59.43|11.83
33|Banas_Finance|50.25|1.38|1.33|138.47|1.51|1.39|93.32|10.03
34|Kiran_Vyapar|168.35|3.13|2.53|525.54|3.07|0.33|146.54|9.97
35|VLS_Finance|188.55|2.68|2.32|589.27|2.02|0.23|272.1|9.94
36|Manaksia|85.55|4.76|0.53|478.51|2.22|0.59|117.84|9.29
37|Jindal_Poly_Inve|282.75|1|0.27|2181.86|3.6|0.13|645.75|8.43
38|Veritas_(India)|137|3.21|0.18|570.48|3.42|0.18|114.32|5.71
39|Natraj_Proteins|138.4|3.46|0.33|41.29|1.94|1.05|15.02|4.47
40|Arihant's_Secur.|13.8|3.73|2.64|6.6|3.49|1.29|1.85|4.4
41|PBM_Polytex|155.65|4.56|0.43|96.25|2.67|0.92|23.5|3.07
42|Rama_Paper_Mills|15.91|2.59|0.16|15.15|1.13|1.87|5.94|1.05
43|MRO-TEK_Realty|62.85|1.49|0.93|181.19|2.02|1.61|78.96|-2.91
44|Pearl_Polymers|23.15|1.49|1.03|34.87|1.06|0.72|26.12|-14.6


In [ ]:
value_investing = pd.read_csv('value_investing_1.txt', delimiter = '|')

In [ ]:
value_investing

In [ ]:
value_investing.drop('S_No', inplace = True, axis = 1)

In [ ]:
value_investing

In [ ]:
value_investing['ev_to_gross_profit'] = value_investing['EV_(Cr)'] / value_investing['Net_Profit_(Cr)']

In [ ]:
value_investing

In [ ]:
metrics = {
            'price_to_earnings': 'PE Percentile',
            'price_to_book_value':'PB Percentile',
            'price_to_sales': 'PS Percentile',
            'ev_to_ebitda':'EV/EBITDA Percentile',
            'ev_to_gross_profit':'EV/GP Percentile'
}

for row in value_investing.index:
    for metric in metrics.keys():
        value_investing.loc[row, metrics[metric]] = score(value_investing[metric], value_investing.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(value_investing[metric])

#Print the entire DataFrame    
value_investing

In [ ]:
from statistics import mean

for row in value_investing.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(value_investing.loc[row, metrics[metric]])
    value_investing.loc[row, 'RV Score'] = mean(value_percentiles)
    
value_investing

In [ ]:
value_investing.sort_values(by = 'RV Score', inplace = True)
value_investing = value_investing[:15]
value_investing.reset_index(drop = True, inplace = True)
value_investing

In [ ]:
portfolio_size = input('Enter the value of your portfolio: ')

In [ ]:
val = float(portfolio_size)
position_size = val/len(final_return_data_hqm.index)
print(position_size)

In [ ]:
for i in range(len(value_investing.index)):
    value_investing.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(value_investing.iloc[i, 1]))
    
value_investing